<font color="magenta" size=7><i>Ant Simulator</i></font>
    

    
This implementation of a probabilistic skeleton enables innate motor control capabilities .  [Main Publication.](https://www.biorxiv.org/content/10.1101/2021.05.18.444689v1)

<div style="text-align:right"><font size=7 color="orchid" face='Brush Script MT'> to Start, Click - <button class="btn btn-sm btn-success"><i class="fa fa-lg fa-id-card-o">

<h3 class="text-center">Abstract</h3>

Genetically encoded structure endows neural networks of the brain with innate computational ca
pabilities that enable odor classification and basic motor control right after birth.
It is also conjectured that the stereotypical laminar organization of neocortical microcircuits
provides basic computing capabilities on which subsequent learning can build.
However, it has remained unknown how nature achieves this.
Insight from artificial neural networks does not help to solve this problem,
since virtually all their computational capabilities result from learning.
We show that genetically encoded control over connection probabilities between different types of neurons suffices for programming substantial computing capabilities into neural networks, providing a plausible mechanism for the evolution of innate capabilities.. This insight also provides a method for enhancing computing and
learning capabilities of artificial neural networks and neuromorphic hardware through
clever initialization.

**Description** 

Probabilistic skeletons (PS) are a mathematical model for encoding innate network structure in 
spiking neural networks.
In this demo we show that a PS can be used to generate spiking networks that can solve a 
quadruped locomotion control task.

**Running the code**

Make sure that the `requirements.txt` are installed.
The model can be run using the command: `python ant_task.py`
The results can be found in: `ant/eval_runs/`. See the <font color="blue">Code</font> section below for a more thorough description of the code.

**Videos**

The videos present results from two different spiking networks generated from the same probabalistic skeleton. We have optimized a probabilistic skeleton on performing a quadrupedal locomotion task. It can be used to sample an arbitrary amount of different spiking neural networks, which can all control the model of the ant. The two sample videos illustrate the performance of two different spiking networks sampled from the same PS. 
The spiking neural networks receive the angles of the ant as well as the height of the torso as input and output torques to the joints to generate a forward movement. 

In [ ]:
from IPython.display import Video

In [ ]:
Video("Data/videos/video.mp4")

In [ ]:
Video("Data/videos/video2.mp4")

# <font color="green"> Visuals

These figures describe the model arhitecture and visualize the inner working of the spiking network. Use the **slider** below to view different figures. Figures and the videos can be found inside of the `visuals` folder.


In [ ]:
from ipywidgets import interact
from IPython.core.display import display, Image, HTML

In [ ]:
figures=!cd Data/figures && ls
@interact(  figure=(0,len(figures)-1) ) 
def displayer( figure ):
    display(HTML("<h3 class='text-center'>"+figures[figure]+"</h3>;&nbsp;&nbsp;"))
    display(Image(filename="Data/figures/"+figures[figure]))
    

`description.png`: System  architecture, indicating network inputs and outputs, 
as well as the 8 joints  that are controlled by the network outputs

`probabilistic_skeleton.png`: Probabilistic skeleton for solving this motor control task. 
Spiking neurons are grouped according to their genetic type. There are both excitatory neurons and inhibitory neurons with different parameters in the neuron model, which both have been fitted to measurement data of biological neurons. 
This PS uses 15 recurrent neuron types, 12 of which are excitatory and 3 are inhibitory. The connectivity of the model is 
defined through the connection probabilities between the types and the spatial distance between pairs of neurons.

`spike_raster_1-3.png`: Spike raster of an RSNN sample with 458 neurons drawn from this probabilistic skeleton. 
The required spatial organization of network outputs emerges through population coding of 9 input variables

`input_output.png`: Sample dynamics of input and output variables of the RSNN controller on a larger timescale

`vid_pca_z_plot.png`: The first row of the figure shows the movement of the ant over time. 
The second row depicts the PCA analysis of the spiking activity of the model. The most recent part of the 
trajectory has been plotted in red. 
Note, that a circular activity pattern emerged, which corresponds to the regular movement pattern of the ant. 
The third row illustrates the spike raster the PCA was based on.

`chord_diagram.png`: The chord diagram depicts the connectivity between the neuron types. 

# <font color="blue">Code

Here we highlight particularly important code in implementing our algorithm.
To understand our implementation we reccomend looking at the code inside of the [ant_task.py](Code/ant_task.py) and [mdels.py](Code/models.py) files. 
[ant_task.py](Code/ant_task.py) contains the `main()` function which is the entrypoint to all code running inside the application. The code for constructing the spiking network and running it is defined inside the [models.py](Code/models.py) file. The two most important functions to read first are the `call` and the `sample_params` function shown below.

## <font color="blue">Call Function

This function simulates the model of the spiking network. The is located inside of the [models.py](http://wetai.gi.ucsc.edu:8007/edit/Projects/Ant_Sim_Haussler_Maass_Collaboration/Code/models.py) file at line `593`. Consecutive calls to this function will computes the activity of the model.

``` python
   def call(self, inputs, state, constants=None):
        batch_size = inputs.shape[0]
        if batch_size is None:
            batch_size = tf.shape(inputs)[0]
        external_current = inputs

        z, v, r, psc_rise, psc = state

        psc_rise = tf.reshape(psc_rise, (batch_size, self.n_glif_neurons, self._n_receptors))
        psc = tf.reshape(psc, (batch_size, self.n_glif_neurons, self._n_receptors))

        z_buf = tf.reshape(z, (self.all_flat_size, self.max_delay, -1))  # self.make_flat_shape(z)
        z = z_buf[:, 0]  # z contains the spike from the prev time step
        z_rec = z_buf[:, 1:]
        inputs = self._compute_internal_currents(z_rec, external_current, batch_size)

        new_psc_rise = self.syn_decay * psc_rise + inputs * self.psc_initial
        new_psc = psc * self.syn_decay + self._dt * self.syn_decay * new_psc_rise

        new_r = tf.nn.relu(r + z * self.t_ref - self._dt)

        input_current = tf.reduce_sum(psc, -1)

        if constants != None:
            input_current = constants[0]

        decayed_v = self.decay * v

        gathered_g = self.param_g * self.e_l

        c1 = input_current + gathered_g
        new_v = decayed_v + self.current_factor * c1

        new_z = spike_function(new_v, self.v_th)
        if self.flags.less_excitable > 0.0:
            excitation_mask = 1 - (1 - self.deletion_mask) * np.random.binomial(1,
                                                                                1 - self.flags.less_excitable,
                                                                                np.shape(self.deletion_mask))
        else:
            excitation_mask = self.deletion_mask
        new_z = excitation_mask * new_z  # apply neuron deletion

        old_new_v = tf.where(z > 0.5, self.v_reset, v)  # v_rec + reset_current_rec
        new_v = tf.where(new_r > 0., old_new_v, new_v)
        new_z = tf.where(new_r > 0., tf.zeros_like(new_z), new_z)

        new_psc = tf.reshape(new_psc, (batch_size, self.n_glif_neurons * self._n_receptors))
        new_psc_rise = tf.reshape(new_psc_rise, (batch_size, self.n_glif_neurons * self._n_receptors))

        # the last neurons are the output neurons
        new_z_out = new_z[..., -self.n_out_neurons:]
        outputs = (new_z_out, dict(v_rec=new_v[..., :-self.n_out_neurons],
                                   v_out=new_v[..., -self.n_out_neurons:],
                                   z_rec=new_z[..., :-self.n_out_neurons],
                                   z_out=new_z_out))

        # add new time step to beginning buffer and drop last
        new_z_buf = tf.concat((new_z[:, None], z_buf[:, :-1]), 1)
        new_z_buf = tf.reshape(new_z_buf, (self.all_flat_size, -1))
        new_state = (new_z_buf, new_v, new_r, new_psc_rise, new_psc)

        return outputs, new_state
```

## <font color="blue">Sample Parameters

The function creates the parameters for the spiking network .  The code is located inside of the [models.py](http://wetai.gi.ucsc.edu:8007/edit/Projects/Ant_Sim_Haussler_Maass_Collaboration/Code/models.py) file at line `293`. One can  sample a spiking network  parameters form from the same probabilistic skeleton. 

``` python
    def sample_params(self):
        # update params
        self.update_params()
        # update spatial structure
        self.update_spatial_structure()
        connection_probabilities, distance_factor = self.compute_connection_probabilities()

        # apply connection constraints mask (no in->out, out->rec and out->out)
        self.connection_parameter *= self.type_mask
        connection_probabilities *= self.type_mask
        if self.flags.spatial_input_to_single_type:
            connection_probabilities *= self.input_restriction_type_mask

        # compute in_weights indices
        # select right section of dist matrix
        in_dist_fact = distance_factor[:, :self.n_in_neurons, self.n_in_neurons:]
        in_weights = self.create_weight_matrix(connection_probabilities, in_dist_fact,
                                               self.in_neuron_type_ids, self.n_in_neurons,
                                               self.glif_neuron_type_ids, self.n_glif_neurons,
                                               n_receptors=self._n_receptors, quantize=True,
                                               position="in") * self.flags.w_in_coeff
        self.in_weights.assign(in_weights)

        # compute rec weights indices
        rec_dist_fact = distance_factor[:, self.n_in_neurons:, self.n_in_neurons:]
        rec_weights = self.create_weight_matrix(connection_probabilities, rec_dist_fact,
                                                self.glif_neuron_type_ids, self.n_glif_neurons,
                                                self.glif_neuron_type_ids, self.n_glif_neurons,
                                                n_receptors=self._n_receptors, use_e_i=True,
                                                quantize=True)
        rec_weights *= self.w_out_coeff_mask
        rec_weights = self.expand_ei_weights(rec_weights)
        self.rec_weights.assign(rec_weights)
```